# Introduction to OSPF
<i>Adapted for use with FABRIC from [OSPF](https://www.cs.unc.edu/Research/geni/geniEdu/06-Ospf.html)</i>
    
In this lab you will experiment with the OSPF routing protocol.

OSPF stands for Open-Shortest-Path-First. OSPF is a dynamic routing protocol which means it can automatically react to unexpected connectivity changes in the network. 

Each OSPF router maintains a Link State Database (LSDB) containing Link State Advertisements (LSAs). 

OSPF routers use Hello messages to discover their neighbors, and determine when links break. By default (and in our example network) OSPF routers send Hello packets every 10 seconds on point to point segments. 

Newly discovered neighbors exchange their LSDBs with Database Description messages (DDs). OSPF routers then fill in the gaps in their databases using a three-way handshake consisting of a Link-State Request (LSR), Link-State Update (LSU), and finally a Link-State Acknowledgement (LSAck). 
OSPF employs a configurable Router Dead Interval, which is the maximum time after which a "silent" router (one that hasn't sent a Hello) is declared "down". 

OSPF uses Dijkstra's Shortest path algorithm to compute routes. 

OSPF allows for hierarchical routing using areas, but to keep things simple, for this module all routers are configured in the same area (area zero). 

You can read more about OSPF and working with FRR’s implementation of it here: https://docs.frrouting.org/en/latest/ospfd.html 

## 1. Set up the Experiment


### 1.1  Retrieve Slice
Import the slice you created in the previous step (with the [Create Slice Notebook](./CreateSlice.ipynb)) here.


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

import json
import traceback

In [ ]:
slice_name = "OSPF"
slice = fablib.get_slice(slice_name)
slice.list_nodes()

### 1.2 Set Routes
Setup connections between nodes on 4 separate networks.

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
#1
try:    
    A = slice.get_node(name="ND_A") 
    B = slice.get_node(name="ND_B")
    C = slice.get_node(name="ND_C")
    D = slice.get_node(name="ND_D")
    
    subnet1 = IPv4Network("10.1.1.0/24")
    subnet2 = IPv4Network("10.1.2.0/24")
    subnet3 = IPv4Network("10.1.3.0/24")
    subnet4 = IPv4Network("10.1.4.0/24")
    
    AP1 = A.get_interface(network_name="Lan1") 
    AP1.ip_addr_add(addr="10.1.1.1", subnet=subnet1)
    
    BP1 = B.get_interface(network_name="Lan1") 
    BP1.ip_addr_add(addr="10.1.1.2", subnet=subnet1)
    
    BP2 = B.get_interface(network_name="Lan2") 
    BP2.ip_addr_add(addr="10.1.2.1", subnet=subnet2)
            
    CP1 = C.get_interface(network_name="Lan2") 
    CP1.ip_addr_add(addr="10.1.2.2", subnet=subnet2)
    
    CP2 = C.get_interface(network_name="Lan3") 
    CP2.ip_addr_add(addr="10.1.3.1", subnet=subnet3)
    
    DP1 = D.get_interface(network_name="Lan3") 
    DP1.ip_addr_add(addr="10.1.3.2", subnet=subnet3)
    
    DP2 = D.get_interface(network_name="Lan4") 
    DP2.ip_addr_add(addr="10.1.4.1", subnet=subnet4)
            
    AP2 = A.get_interface(network_name="Lan4") 
    AP2.ip_addr_add(addr="10.1.4.2", subnet=subnet4)
except Exception as e:
    print(f"Exception: {e}")

### 1.3 Installation Script
Upload and execute installation script on all nodes. The OSPF and Zebra services are then started.
You can read through the installation script [Here](./scripts/frr-script.sh).
After the script runs, we turn on the link interfaces so the nodes can communicate

In [ ]:
for node in slice.get_nodes():
    
    node.upload_file("scripts/frr-script.sh","frr-script.sh")
    node.execute("chmod +x frr-script.sh")
    node.execute("./frr-script.sh")
    stdout, stderr = node.execute(f'sudo ip link set dev enp7s0 up')
    stdout, stderr = node.execute(f'sudo ip link set dev enp8s0 up')
    
    


## 2. Run Experiment

### 2.1 Router Interface

FRR provides an integrated user interface shell called vtysh, which connects to each of the underlying daemons (zebra and ospfd in our case). vtysh is a Cisco-like command line interface (CLI). Many of the commands are similar to Cisco IOS (Internetwork Operating System) commands. With frr installed, each of your nodes functions like a router, and in vtysh your shell will operate like a router configuration interface. To open a vtysh shell on each of your nodes, do the following for each node:

1. Open a new terminal (click the blue + button in the top-left side of the JupyterHub window to open a new launcher, then click "Terminal" under "Other" in that launcher window)

2. SSH into the node (use the SSH Command shown for that node in the output of step 1.1 of this notebook)

3. Open a vtysh shell by typing: `sudo vtysh`

4. Look at the OSPF neighbors for Node A by typing the following in the vtysh console for node A:  
    ```
    show ip ospf neighbor
    ```
    You should see something similar to: 
    ```
    Neighbor ID     Pri State           Up Time         Dead Time Address         Interface                        RXmtL RqstL DBsmL
    10.1.2.1          1 Full/Backup     2m35s             32.948s 10.1.1.2        enp7s0:10.1.1.1                      0     0     0
    10.1.4.1          1 Full/Backup     1m18s             32.289s 10.1.4.1        enp8s0:10.1.4.2                      0     0     0
    ```
    This shows that node A has found 2 neighbors, one with ID 10.1.4.1 (node D), and one with ID 10.1.2.1 (node B). The state column indicates the adjacency state. Full indicates that all the router and network LSAs have been exchanged and the routers' databases are fully synchronized. 

    DR indicates that this is the Designated Router, while Backup indicates this is the Backup Designated Router (BDR). 

    Dead Time shows the countdown of the Dead Interval. It is reset to the Dead Interval each time a hello message is received. 

    The Interface column indicates which network interface the neighbor is directly connected to. 

5. Look at the routing table for node A by typing the following in the console: 
    ```
    show ip ospf route
    ```
    This should produce output similar to: 
    ```
    ============ OSPF network routing table ============
    N    10.1.1.0/24           [1] area: 0.0.0.0
                               directly attached to enp7s0
    N    10.1.2.0/24           [2] area: 0.0.0.0
                               via 10.1.1.2, enp7s0
    N    10.1.3.0/24           [2] area: 0.0.0.0
                               via 10.1.4.1, enp8s0
    N    10.1.4.0/24           [1] area: 0.0.0.0
                               directly attached to enp8s0

    ============ OSPF router routing table =============

    ============ OSPF external routing table ===========
    ```
    This routing table shows that from the given node, to get to the networks indicated in the 2nd column, a packet must be forwarded through the interface indicated in the third column, or forwarded to the directly attached interface indicated in the third column. It may help to look back at the network diagram to verify the values in the table. For example, to get from node A to the 10.1.2.0 network, a packet should be routed through 10.1.1.2 (B's interface to A). On the other hand, the 10.1.4.0 network is directly attached to a local ethernet interface (enp8s0 in this example). Packets destined for this network should be routed out that interface.

    **Copy your output for Node A’s routing table into Lab03_Worksheet.docx**

    From the diagram, we can see that Node A has 2 possible routes to Node C: one through Node B, and one through Node D. We can also see that Node C has 2 interfaces: 10.1.3.1 and 10.1.2.2. Based on the forwarding table, which route would Node A use to reach C’s 10.1.3.1 interface? What about the 10.1.2.2 interface? How can you tell? **Answer in Lab03_Worksheet.docx**

Repeat steps 4-5 above for Node C: **Copy your output for Node C’s routing table into Lab03_Worksheet.docx**

Node C has 2 possible routes to Node A: one through Node B, and one through Node D. Based on the forwarding table, which one would it use to reach Node A’s 10.1.1.1 interface? How can you tell? **Answer in Lab03_Worksheet.docx**

### 2.2 Dead Link
1. Exit out of the vtysh shell on **node D's SSH Terminal** by typing: ```exit```

2. Determine which interface is associated with IP address 10.1.4.1 by typing: ```ip addr``` and finding the interface (i.e. enp7s0 or enp8s0) that lists 10.1.4.1 as its IP address.

3. Take down that interface using the following command: 
    ```
    sudo ip link set dev <interface> down
    ```
    where `<interface>` is replaced with the interface identifier found in the previous step. 

4. Quickly switch back to the vtysh shell on **node A's SSH Terminal**, and re-issue the following command to view the neighbor table: 
    ```
    show ip ospf neighbor
    ```

5. Press up arrow, Enter, repeatedly to re-issue the command multiple times as you watch the Dead Time count down, until the Neighbor is removed from the table. Notice that the other neighbor in the table begins to count down from 40, but gets reset to 40 again each time it receives a Hello message.

    **Copy the output for Node A’s neighbor table that shows Node D has been removed into Lab03_Worksheet.docx**

6. Back on **node D's SSH Terminal**, bring the interface back up by typing: 
    ```
    sudo ip link set dev <interface> up
    ```

7. On **node A's SSH Terminal**, and re-issue the following command: 
    ```
    show ip ospf neighbor
    ```
    Notice the neighbor is back in the table. **Copy the output that shows this into Lab03_Worksheet.docx**. You may also be able to watch the State of the newly added neighbor change from Init or Loading to Full. This is an example of how OSPF uses hello messages to update the neighbor table as network connections change over time. 

### 2.3 Route Change

1. Traceroute is a tool for determining the route packets takes through the network from one node to another. On **node A's SSH Terminal**, trace the route from A to C's 10.1.3.1 interface by typing:
    ```
    traceroute 10.1.3.1
    ```
    Note that occasionally traceroute may fail. If ever this happens, and it takes more than a few seconds to complete or you see "* * *", type Ctrl-c, and re-issue the command. 

2. The output of the command should look similar to the following: 
    ```
    traceroute to 10.1.3.1 (10.1.3.1), 30 hops max, 60 byte packets
     1  10.1.4.1 (10.1.4.1)  0.058 ms  0.048 ms  0.043 ms
     2  10.1.3.1 (10.1.3.1)  0.117 ms  0.112 ms  0.125 ms
    ```
    This shows that a packet leaving A and heading to C will travel first to 10.1.4.1 (on node D), and then to 10.1.3.1 (on node C). This shows the route is going from A through D to C.

3. As before, take down one of node D's interfaces. On **node D's SSH Terminal** take down the interface corresponding to the 10.1.4.1 address using the same following command: 
```
ip link set dev <interface> down
```

4. Re-run the traceroute on node A's SSH Terminal by typing:
```
traceroute 10.1.3.1 
```

5. Packets should be automatically rerouted through Node B. Can you tell that this is happening? How? **Answer and copy and paste your output in Lab03_Worksheet.docx**

6. Back on **node D's SSH Terminal**, bring the interface back up by typing:
```
ip link set dev <interface> up
```

7. Issue the traceroute command on **Node A's SSH Terminal**, multiple times, and notice that eventually the route reverts to the original route through node D. **Copy and paste the output that shows this in Lab03_Worksheet.docx**

## F. Link Costs 

1. By default, OSPF link costs are determined based on the link’s bandwidth. Specifically, under the default settings, links with bandwidth of 100 Mbit/s or more have cost 1, and links with lower bandwidth have cost = (100 Mbit/s / link_bandwidth). You can view the cost for each of a node’s interfaces by typing the following in vtysh:
    ```
    show ip ospf interface 
    ```
    This will list information about all of the node’s interfaces (for us, enp7s0 and enp8s0). The output should look something like:
```
NDB# show ip ospf interface
enp7s0 is up
  ifindex 3, MTU 1500 bytes, BW 100000 Mbit <UP,BROADCAST,RUNNING,MULTICAST>
  Internet Address 10.1.2.1/24, Broadcast 10.1.2.255, Area 0.0.0.0
  MTU mismatch detection: enabled
  Router ID 10.1.2.1, Network Type BROADCAST, Cost: 1
  Transmit Delay is 1 sec, State DR, Priority 1
  Designated Router (ID) 10.1.2.1 Interface Address 10.1.2.1/24
  Backup Designated Router (ID) 10.1.3.1, Interface Address 10.1.2.2
  Saved Network-LSA sequence number 0x80000002
  Multicast group memberships: OSPFAllRouters OSPFDesignatedRouters
  Timer intervals configured, Hello 10s, Dead 40s, Wait 40s, Retransmit 5
    Hello due in 0.492s
  Neighbor Count is 1, Adjacent neighbor count is 1
  Graceful Restart hello delay: 10s
enp8s0 is up
  ifindex 4, MTU 1500 bytes, BW 100000 Mbit <UP,BROADCAST,RUNNING,MULTICAST>
  Internet Address 10.1.1.2/24, Broadcast 10.1.1.255, Area 0.0.0.0
  MTU mismatch detection: enabled
  Router ID 10.1.2.1, Network Type BROADCAST, Cost: 1
  Transmit Delay is 1 sec, State Backup, Priority 1
  Designated Router (ID) 10.1.1.1 Interface Address 10.1.1.1/24
  Backup Designated Router (ID) 10.1.2.1, Interface Address 10.1.1.2
  Multicast group memberships: OSPFAllRouters OSPFDesignatedRouters
  Timer intervals configured, Hello 10s, Dead 40s, Wait 40s, Retransmit 5
    Hello due in 2.353s
  Neighbor Count is 1, Adjacent neighbor count is 1
  Graceful Restart hello delay: 10s
```
    Notice that the second line for each interface (starting with “Internet Address”) gives the IP address for the interface, and the fourth line (starting with “Router ID”) gives the cost. **Run this command on Node A and record the IP address and cost for each interface in Lab03_Worksheet.docx.**

2. In the Route Change section above, we saw that normally Node A will route through Node D to reach Node C’s 10.1.3.1 interface. But, as we learned in class, routing decisions are made based on link costs. **How could you modify the cost for one of Node A’s interfaces such that it will normally route to the 10.1.3.1 interface by going through Node B? Answer in Lab03_Worksheet.docx (and be specific: which interface would you change, and what value would you set its cost to?)**

3. In vtysh, you can set the cost for an interface from the configuration mode. First, type `config` to enter the general configuration mode, then type `interface <interface>` (with `<interface>` replaced by the name of the interface you want to modify, enp7s0 or enp8s0) to enter the interface-specific configuration. Then, you can use the command `ip ospf cost <cost>` (with `<cost>` replaced by the cost you want to set for the interface) to modify the cost. Then type `exit` twice to exit the configuration modes. For example, to set the cost for enp7s0 to 10 on Node C, this session would look like:
    ```
    NDC# config
    NDC(config)# interface enp7s0
    NDC(config-if)# ip ospf cost 10
    NDC(config-if)# exit
    NDC(config)# exit
    ```
    Use this approach to implement the change you described in step 2 above on Node A. **Copy and paste the commands into Lab03_Worksheet.docx.**

4. Re-run the traceroute to Node C on Node A's SSH Terminal by typing:
    ```
    traceroute 10.1.3.1 
    ```
    Did you succeed in getting the traffic to flow through Node B? **Answer and copy and paste the output into Lab03_Worksheet.docx.**


## Cleanup Resources

Once you have completed the lab, shut down the slice.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
import json
import traceback

slice_name = "OSPF"
try:
    slice = fablib.get_slice(slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")